# Loop 1 LB Feedback Analysis

## Current Status
- **Best CV/LB Score**: 70.7343
- **Target**: 68.931058
- **Gap**: 1.803 points (2.6%)

## Key Observations
1. CV = LB (perfect match) - this is expected for deterministic optimization problems
2. The pre-optimized submission is already at a local optimum
3. bbox3 binary requires GLIBC 2.34 (system has 2.31) - cannot use directly
4. tree_packer_v21 C++ source is available and can be compiled

In [ ]:
import pandas as pd
import numpy as np
import os

# Load the current best submission
submission_path = '/home/code/experiments/001_baseline/submission.csv'
df = pd.read_csv(submission_path)
print(f'Submission shape: {df.shape}')
print(df.head(10))

In [ ]:
# Analyze per-N scores to identify worst performers
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import math

getcontext().prec = 30
scale_factor = Decimal('1e18')

# Tree template
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def score_group(xs, ys, degs):
    n = len(xs)
    V = len(TX)
    mnx, mny = 1e300, 1e300
    mxx, mxy = -1e300, -1e300
    
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        xi, yi = xs[i], ys[i]
        for j in range(V):
            X = c * TX[j] - s * TY[j] + xi
            Y = s * TX[j] + c * TY[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    
    side = max(mxx - mnx, mxy - mny)
    return side * side / n, side

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

# Parse submission
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

per_n_scores = []
for n, g in df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc, side = score_group(xs, ys, ds)
    per_n_scores.append({'N': n, 'score': sc, 'side': side, 'contribution_pct': 0})

per_n_df = pd.DataFrame(per_n_scores)
total_score = per_n_df['score'].sum()
per_n_df['contribution_pct'] = per_n_df['score'] / total_score * 100

print(f'Total score: {total_score:.10f}')
print(f'\nTop 20 worst N values (highest contribution):')
print(per_n_df.nlargest(20, 'score')[['N', 'score', 'side', 'contribution_pct']].to_string())

In [ ]:
# Analyze score distribution
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Score by N
ax1 = axes[0, 0]
ax1.bar(per_n_df['N'], per_n_df['score'], alpha=0.7)
ax1.set_xlabel('N')
ax1.set_ylabel('Score (s²/n)')
ax1.set_title('Score Contribution by N')

# Side length by N
ax2 = axes[0, 1]
ax2.scatter(per_n_df['N'], per_n_df['side'], alpha=0.5, s=10)
ax2.set_xlabel('N')
ax2.set_ylabel('Side Length')
ax2.set_title('Bounding Box Side Length by N')

# Cumulative contribution
ax3 = axes[1, 0]
sorted_df = per_n_df.sort_values('score', ascending=False)
sorted_df['cumsum'] = sorted_df['contribution_pct'].cumsum()
ax3.plot(range(1, 201), sorted_df['cumsum'].values)
ax3.axhline(y=50, color='r', linestyle='--', label='50%')
ax3.axhline(y=80, color='g', linestyle='--', label='80%')
ax3.set_xlabel('Number of N values (sorted by contribution)')
ax3.set_ylabel('Cumulative % of Total Score')
ax3.set_title('Cumulative Score Contribution')
ax3.legend()

# Efficiency (side / sqrt(N))
ax4 = axes[1, 1]
per_n_df['efficiency'] = per_n_df['side'] / np.sqrt(per_n_df['N'])
ax4.scatter(per_n_df['N'], per_n_df['efficiency'], alpha=0.5, s=10)
ax4.set_xlabel('N')
ax4.set_ylabel('Efficiency (side / √N)')
ax4.set_title('Packing Efficiency by N')

plt.tight_layout()
plt.savefig('/home/code/exploration/per_n_analysis.png', dpi=100)
plt.show()

print(f'\nN values with worst efficiency (highest side/√N):')
print(per_n_df.nlargest(10, 'efficiency')[['N', 'side', 'efficiency', 'score']].to_string())

In [ ]:
# Calculate how much improvement is needed per N to reach target
target = 68.931058
current = total_score
gap = current - target

print(f'Current score: {current:.6f}')
print(f'Target score: {target:.6f}')
print(f'Gap to close: {gap:.6f} ({gap/current*100:.2f}%)')

# If we improve all N equally by X%, what X do we need?
# new_score = current * (1 - X/100)
# target = current * (1 - X/100)
# X = (1 - target/current) * 100
required_improvement_pct = (1 - target/current) * 100
print(f'\nRequired uniform improvement: {required_improvement_pct:.2f}%')

# Alternatively, if we focus on worst N values
worst_20 = per_n_df.nlargest(20, 'score')
worst_20_total = worst_20['score'].sum()
print(f'\nWorst 20 N values contribute: {worst_20_total:.6f} ({worst_20_total/current*100:.1f}%)')
print(f'If we improve worst 20 by 10%: saves {worst_20_total * 0.1:.6f}')
print(f'If we improve worst 20 by 20%: saves {worst_20_total * 0.2:.6f}')

In [ ]:
# Check what datasets/submissions are available for ensemble
import os
import glob

print('Available datasets:')
for d in glob.glob('/home/code/datasets/*'):
    print(f'  {d}')
    for f in glob.glob(f'{d}/*.csv'):
        print(f'    - {os.path.basename(f)}')

print('\nAvailable submission files:')
for f in glob.glob('/home/code/**/*.csv', recursive=True):
    if 'submission' in f.lower():
        print(f'  {f}')

In [ ]:
# Key insight: We need to implement ENSEMBLE approach
# The jonathanchan kernel shows that combining best N from multiple sources
# is the key to getting good scores

# Let's compare our current submission with the bucket-of-chump submission
boc_path = '/home/code/datasets/bucket-of-chump/submission.csv'
santa_path = '/home/code/datasets/santa-2025-csv/santa-2025.csv'

def score_submission(path):
    df = pd.read_csv(path)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    scores = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        sc, side = score_group(xs, ys, ds)
        scores[n] = {'score': sc, 'side': side}
    return scores, sum(s['score'] for s in scores.values())

boc_scores, boc_total = score_submission(boc_path)
santa_scores, santa_total = score_submission(santa_path)
our_scores = {row['N']: {'score': row['score'], 'side': row['side']} for _, row in per_n_df.iterrows()}

print(f'bucket-of-chump total: {boc_total:.10f}')
print(f'santa-2025-csv total: {santa_total:.10f}')
print(f'Our current total: {total_score:.10f}')

# Find N values where other submissions are better
better_in_boc = []
better_in_santa = []
for n in range(1, 201):
    our = our_scores[n]['score']
    boc = boc_scores[n]['score']
    santa = santa_scores[n]['score']
    if boc < our - 1e-12:
        better_in_boc.append((n, our - boc))
    if santa < our - 1e-12:
        better_in_santa.append((n, our - santa))

print(f'\nN values where bucket-of-chump is better: {len(better_in_boc)}')
if better_in_boc:
    print(f'  Potential improvement: {sum(x[1] for x in better_in_boc):.10f}')
    print(f'  Top 5: {sorted(better_in_boc, key=lambda x: -x[1])[:5]}')

print(f'\nN values where santa-2025-csv is better: {len(better_in_santa)}')
if better_in_santa:
    print(f'  Potential improvement: {sum(x[1] for x in better_in_santa):.10f}')
    print(f'  Top 5: {sorted(better_in_santa, key=lambda x: -x[1])[:5]}')

## Key Findings

1. **CV = LB perfectly** - Expected for deterministic optimization
2. **Gap is 2.6%** - Need ~1.8 points improvement
3. **Small N values contribute most** - N=1 to N=20 are critical
4. **Ensemble is key** - Combining best N from multiple sources
5. **bbox3 unavailable** - Must use tree_packer_v21 or implement SA in Python

## Recommended Strategy

1. **Implement ensemble approach** - Combine best configurations from multiple public submissions
2. **Run extended tree_packer_v21** - With higher iterations (-n 10000 -r 100+)
3. **Implement fractional translation** - Very fine position adjustments
4. **Focus on worst N values** - Targeted optimization
5. **Download more public submissions** - More sources = better ensemble